# Backup

**Sources:**

- [`rsync` usage example](https://www.google.com/search?channel=fs&client=ubuntu-sn&q=how+to+linux+backup&bshm=nce/1#fpstate=ive&vld=cid:d262cccb,vid:GqSxR93xK6E).
- [`rsync` documentation](https://download.samba.org/pub/rsync/rsync.1).
- [Backup with `rsync`](https://jumpcloud.com/blog/how-to-backup-linux-system-rsync);
- [Official ubuntu backup instructions](https://help.ubuntu.com/community/BackupPC);
- [Offical ubutntu backup instructions - section about `tar`](https://help.ubuntu.com/community/BackupYourSystem/TAR).

## `tar`

In the following cell I just try to follow the instructions from <a href="https://help.ubuntu.com/community/BackupYourSystem/TAR">official ubutntu backup instructions - section about `tar`</a>, to make a backup - and test how it works in different cases.

I want to try:

- **step 1 - backup creating**:
    - run docker container with ubuntu;
    - install `curl`, `abache2`;
    - run the `apache2` service;
    - check `apache2` with curl;
    - use `tar` to make a backup of the current system;
    - stop the container;
- **step2 - recovery from back up**:
    - run a new empty ubuntu container;
    - copy the backup archive from the previous container;
    - restore the backup in empty ubuntu system;
    - check `curl` and `apache2`.

**step 1 - backup creating**

In [70]:
%%bash
mkdir backups
docker run --rm --name ubun_cont -i -v $(pwd)/backups:/backups ubuntu

echo "=====install====="
apt-get update &> /dev/null
apt-get install curl apache2 -y &> /dev/null
service apache2 start &> /dev/null


echo
echo "=====check====="
curl -s localhost:80 | head -n 20

echo
echo "=====make backup====="
tar \
    -cvpzf \
    /backups/backup.tar.gz \
    --exclude=/backups/backup.tar.gz \
    --one-file-system / &> /dev/null
exit

=====install=====

=====check=====
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
  <!--
    Modified from the Debian original for Ubuntu
    Last updated: 2022-03-22
    See: https://launchpad.net/bugs/1966004
  -->
  <head>
    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
    <title>Apache2 Ubuntu Default Page: It works</title>
    <style type="text/css" media="screen">
  * {
    margin: 0px 0px 0px 0px;
    padding: 0px 0px 0px 0px;
  }

  body, html {
    padding: 3px 3px 3px 3px;

    background-color: #D8DBE2;

=====make backup=====


There is not much to say - it saves the backup in the "backups" folder.

**step2 - recovery from backup** 

In [71]:
%%bash
docker run --rm --name ubun_cont -i -v $(pwd)/backups:/backups ubuntu
tar -xvpzf /backups/backup.tar.gz -C / --numeric-owner &> /dev/null

echo "=====culr --help====="
curl --help
echo "=====curl localhost:80====="
culr -s localhost:80

echo
echo "=====curl localhost:80 after apache====="
service apache2 start
curl -s localhost:80 | head -n 20

exit

=====culr --help=====
Usage: curl [options...] <url>
 -d, --data <data>          HTTP POST data
 -f, --fail                 Fail silently (no output at all) on HTTP errors
 -h, --help <category>      Get help for commands
 -i, --include              Include protocol response headers in the output
 -o, --output <file>        Write to file instead of stdout
 -O, --remote-name          Write output to a file named as the remote file
 -s, --silent               Silent mode
 -T, --upload-file <file>   Transfer local FILE to destination
 -u, --user <user:password> Server user and password
 -A, --user-agent <name>    Send User-Agent <name> to server
 -v, --verbose              Make the operation more talkative
 -V, --version              Show version number and quit

This is not the full help, this menu is stripped into categories.
Use "--help category" to get an overview of all categories.
For all options use the manual or "--help all".
=====curl localhost:80=====

=====curl localhost:80 aft

/bin/bash: line 6: culr: command not found
AH00558: apache2: Could not reliably determine the server's fully qualified domain name, using 172.17.0.2. Set the 'ServerName' directive globally to suppress this message


 * 
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
  <!--
    Modified from the Debian original for Ubuntu
    Last updated: 2022-03-22
    See: https://launchpad.net/bugs/1966004
  -->
  <head>
    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
    <title>Apache2 Ubuntu Default Page: It works</title>
    <style type="text/css" media="screen">
  * {
    margin: 0px 0px 0px 0px;
    padding: 0px 0px 0px 0px;
  }

  body, html {
    padding: 3px 3px 3px 3px;

    background-color: #D8DBE2;


**Note** Even without downloading `curl` and `apache2` I can use it after unpacking. To use `apache2` you had to start the corresponding service - but it was still really easy.

## `rsync` basics

Actually `rsync` is just a copy tool, but if you compare it with `cp` it has much more options and features.

### Basic syntax

`rsync <copied file> <destination of copying>`

Example follows

In [7]:
%%bash
# creating to directories
mkdir dir1 dir2
# creating file in dir1
echo "hello new file" > dir1/test_file
# rsynk file from dir1 to dir2
rsync dir1/test_file dir2/test_file
# check the message
cat dir2/test_file

rm -r dir1 dir2

hello new file


### `r` - copy *r*ecursive

Allow to copy a folder with all its contents.

In the following example I'm trying to copy contents from dir1 to dir2, as you can see I couldn't do it without the `r` option.

In [22]:
%%bash

mkdir dir1 dir2
touch dir1/file{1..5}.txt

echo "=====content of dir1====="
ls dir1

echo "=====no r option====="
rsync dir1/ dir2
ls dir2

echo "=====r option====="
rsync -r dir1/ dir2
ls dir2

rm -r dir1 dir2

=====content of dir1=====
file1.txt
file2.txt
file3.txt
file4.txt
file5.txt
=====no r option=====
skipping directory .
=====r option=====
file1.txt
file2.txt
file3.txt
file4.txt
file5.txt


### `-a` - attributes

It works just like `r`, but also deals with file attributes (like creation time, user and so on). So in the following example:

- Few files was created in `dir1`;
- The user for these files has been changed from `root` to `user1`;
- Then I use `rsync` twice:
    - First with the `r` option, which puts the user back in `root`;
    - Second, with the `a` option, which saves the user attribute as `user1` from source.

In [38]:
%%bash
# docker container for usperuser access
docker run --rm --name test_container -i ubuntu
apt-get update &> /dev/null
apt-get install -y rsync &> /dev/null
# creating file with some specific user
useradd user1
mkdir dir1 dir2
touch dir1/file{1..5}.txt
chown user1 dir1/file*

echo "=====dir1====="
ls -l dir1

echo
echo "=====dir2 after rsync -r====="
rsync -r dir1/ dir2
ls -l dir2

echo
echo "=====dir2 after rsync -a====="
rsync -a dir1/ dir2
ls -l dir2

exit

=====dir1=====
total 0
-rw-r--r-- 1 user1 root 0 Jun 17 15:31 file1.txt
-rw-r--r-- 1 user1 root 0 Jun 17 15:31 file2.txt
-rw-r--r-- 1 user1 root 0 Jun 17 15:31 file3.txt
-rw-r--r-- 1 user1 root 0 Jun 17 15:31 file4.txt
-rw-r--r-- 1 user1 root 0 Jun 17 15:31 file5.txt

=====dir2 after rsync -r=====
total 0
-rw-r--r-- 1 root root 0 Jun 17 15:31 file1.txt
-rw-r--r-- 1 root root 0 Jun 17 15:31 file2.txt
-rw-r--r-- 1 root root 0 Jun 17 15:31 file3.txt
-rw-r--r-- 1 root root 0 Jun 17 15:31 file4.txt
-rw-r--r-- 1 root root 0 Jun 17 15:31 file5.txt

=====dir2 after rsync -a=====
total 0
-rw-r--r-- 1 user1 root 0 Jun 17 15:31 file1.txt
-rw-r--r-- 1 user1 root 0 Jun 17 15:31 file2.txt
-rw-r--r-- 1 user1 root 0 Jun 17 15:31 file3.txt
-rw-r--r-- 1 user1 root 0 Jun 17 15:31 file4.txt
-rw-r--r-- 1 user1 root 0 Jun 17 15:31 file5.txt


### `--delete` - delete extraneous

If you don't use this option `rsync` will save files wich are in destination directory. Оtherwise all files that are not in the source will be deleted in the destination folder. The following exampe show:
- In `dir2` I created file `test.txt`;
- First I rsync files from `dir1` to `dir2` without the `--delete` option - `test.txt` is still in `dir2`;
- Second I rsync files from `dir1` to `dir2` using the `--delete` option - `test.txt` disappears from `dir2`.

In [6]:
%%bash
mkdir dir1 dir2
touch dir1/file{1..5}.txt
touch dir2/test.txt

echo "====initial dir2====="
ls dir2

echo "=====dir2 after rsync no --delete====="
rsync -r dir1/ dir2
ls dir2

echo "=====dir2 after rsync with --delete====="
rsync -r --delete dir1/ dir2
ls dir2

rm -r dir1 dir2

====initial dir2=====
test.txt
=====dir2 after rsync no --delete=====
file1.txt
file2.txt
file3.txt
file4.txt
file5.txt
test.txt
=====dir2 after rsync with --delete=====
file1.txt
file2.txt
file3.txt
file4.txt
file5.txt
